In [1]:
from sub_overlap_pipeline import *

In [2]:
spark_session = start_session()
df_raw = load_data("sample_data.json", spark_session)

23/03/16 23:22:12 WARN Utils: Your hostname, phy-akre resolves to a loopback address: 127.0.1.1; using 192.168.0.137 instead (on interface wlp2s0)
23/03/16 23:22:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/16 23:22:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_raw.head()

Row(author='Dethcola', author_cakeday=None, author_flair_css_class='', author_flair_text='Clairemont', body='A quarry', can_gild=True, controversiality=0, created_utc=1506816000, distinguished=None, edited='false', gilded=0, id='dnqik14', is_submitter=False, link_id='t3_73ieyz', parent_id='t3_73ieyz', permalink='/r/sandiego/comments/73ieyz/best_place_for_granite_counter_tops/dnqik14/', retrieved_on=1509189606, score=3, stickied=False, subreddit='sandiego', subreddit_id='t5_2qq2q')

In [4]:
df_raw.columns

['author',
 'author_cakeday',
 'author_flair_css_class',
 'author_flair_text',
 'body',
 'can_gild',
 'controversiality',
 'created_utc',
 'distinguished',
 'edited',
 'gilded',
 'id',
 'is_submitter',
 'link_id',
 'parent_id',
 'permalink',
 'retrieved_on',
 'score',
 'stickied',
 'subreddit',
 'subreddit_id']

In [5]:
# Step 2: Filter out columns that will not be used
df_reddit = filter_columns(df_raw)

In [6]:
df_reddit.columns

['author', 'subreddit', 'subreddit_id']

In [7]:
df_reddit.head()

Row(author='Dethcola', subreddit='sandiego', subreddit_id='t5_2qq2q')

In [8]:
subs_to_incl = 100
df_sub_filtered = filter_top_subreddits(df_reddit, subs_to_incl)

In [9]:
df_sub_filtered

DataFrame[author: string, subreddit: string, subreddit_id: string]

In [10]:
# Step 4: Get list with users with significant amount of posts and filter dataframe
comment_threshold = 1
# df_user_filtered = filter_top_users(df_sub_filtered, comment_threshold)

In [11]:
df_user_count = df_sub_filtered.groupBy("author").count()
df_top_users = df_user_count.filter(col("count") > comment_threshold)

In [15]:
df_top_users.select("author")

DataFrame[author: string]

In [13]:
df_top_users.select("author").rdd.flatMap(lambda x: x)

Traceback (most recent call last):
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 565, in _function_reduce
    return self._dynamic_function_red

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [18]:
top_users = df_top_users.select("author").rdd.flatMap(lambda x: x).collect()

Traceback (most recent call last):
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/adeel/anaconda/envs/pyspark/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 565, in _function_reduce
    return self._dynamic_function_red

PicklingError: Could not serialize object: IndexError: tuple index out of range